In [ ]:
import pandas as pd

# Data Import

## Documents preparation

### Arguments reading 

In [ ]:
args_json = pd.read_json("crawlers/procon/procon.json")

def format_args_row(row):
    df = row.apply(pd.Series).T
    df.url = df.url[0]
    df.topic = df.topic[0]
    return(df)

arg_df_series = args_json.apply(format_args_row, axis=1)
args_df = pd.concat(arg_df_series.values.tolist(),ignore_index=True)

In [ ]:
def orientation(pro_con):
    if "Pro" in pro_con:
        return "Pro"
    return "Con"

args_df = args_df.assign(argument_orient = args_df.apply(lambda row: orientation(row.pro_con), axis=1))

### Documents reading

#### Convert from json to dataframe and add the topic and the source stance information 

In [ ]:
import glob
import json

def create_df(path_list, topic, source_stance):
    glob_data = []
    for path in path_list:
        for file in glob.glob(path):
            with open(file) as json_file:
                data = json.load(json_file)
                glob_data.append(data)

        df = pd.DataFrame(glob_data)
        df['text_topic'] = topic
        df['text_source_stance'] = source_stance
    return(df)

In [ ]:
path_list = ["../../news-please-repo/data/2020/02/06/abort73.com/*.json"]
topic = "Abortion"
source_stance = "Con"

abort73 = create_df(path_list, topic, source_stance)

In [ ]:
path_list = ["../../news-please-repo/data/2020/02/06/mpp.org/*.json","../../news-please-repo/data/2020/02/06/blog.mpp.org/*.json"]
topic = "Marijuana"
source_stance = "Pro"

mpp = create_df(path_list, topic, source_stance)

In [ ]:
path_list = ["../../news-please-repo/data/2020/02/06/prochoiceamerica.org/*.json"]
topic = "Abortion"
source_stance = "Pro"

prochoice = create_df(path_list, topic, source_stance)

In [ ]:
path_list = ["../../news-please-repo/data/2020/02/06/calmca.org/*.json"]
topic = "Marijuana"
source_stance = "Con"

calmca = create_df(path_list, topic, source_stance)

In [ ]:
document_df = pd.concat([abort73, prochoice, mpp, calmca], ignore_index=True)

#### Filter the columns of dataframes to use 

In [ ]:
columns = ['authors', 'date_modify', 'date_publish', 'title', 'source_domain', 'text', 'url', 'text_topic', 'text_source_stance']
document_df = pd.DataFrame(document_df, columns=columns)
document_df = document_df.rename(columns={"authors":"text_authors", "title":"text_title", "source_domain":"text_source_domain", "url":"text_url"})

### Preprocessing document's and argument's text

####  Removing rows which text contains None from the data

In [ ]:
args_df = args_df[~args_df.argument.isna()]
document_df = document_df[~document_df.text.isna()]

#### Removing stopwords and measuring the resulting text length 

In [ ]:
from nltk.corpus import stopwords
from nltk import download
download('stopwords') 

stop_words = stopwords.words('english')

def preprocess_sentence(str_array):
    new_sentences = []
    sentences_size = []
    for text in str_array:
        new_sentence = ""
        for word in text.split():
            if word not in stop_words:
                new_sentence += " " + word.lower()
        sentences_size.append(new_sentence.count(" "))
        new_sentences.append(new_sentence.strip())
        
    return(new_sentences, sentences_size)

In [ ]:
args_df['argument_processed'], args_df['argument_processed_size']  = preprocess_sentence(args_df.argument)
document_df['text_processed'], document_df['text_processed_size'] = preprocess_sentence(document_df.text)

### Merge documents and arguments by topic 

In [ ]:
args_docs = args_df.merge(document_df, left_on=['topic'], right_on=['text_topic'])

## Paragraphs preparation

In [ ]:
wiki_json = pd.read_json("crawlers/procon/wiki.json")

In [ ]:
# TODO: Add preprocess_sentence in the paragraphs processing
def format_wiki_row(row):
    df = row.apply(pd.Series).T
    df.source = df.source[0]
    df.title = df.title[0]
    df.topic = df.topic[0]
    df.stance = df.stance[0]
    df.url = df.url[0]
    return(df)

wiki_df_series = wiki_json.apply(format_wiki_row, axis=1)
wiki_df = pd.concat(wiki_df_series.values.tolist(),ignore_index=True)
wiki_df = wiki_df.rename(columns={"topic":"text_topic", "title":"text_title", "content":"text", "source":"text_source_domain", "stance":"text_source_stance", "url":"text_url"})

In [ ]:
args_prg = args_df.merge(wiki_df, left_on=['topic'], right_on=['text_topic'])

## Debate Reasons Dataset Preparation 

In [ ]:

def create_debate_df(path_list, topic):
    glob_data = {}
    for path in path_list:
        for file in glob.glob(path):
            with open(file, encoding = "ISO-8859-1") as fp:
                for line in fp:
                    if line.startswith('Label##'):
                        stance = "Pro"
                        stance_line = line.replace("Label##","").split("-")[0]
                        if stance_line == "c":
                            stance = "Con"                        
                    if line.startswith('Line##'):
                        arg = line.replace("Line##","")
                        if file not in glob_data.keys():
                            glob_data[file] = [topic,stance,stance,arg]
                        else:
                            glob_data[file][3] += (arg.strip() + " ").strip()
    
    dr = pd.DataFrame.from_dict(glob_data, orient='index', columns=["text_topic","text_source_stance","text_source_domain","text"])
    dr["text_title"] = dr.index.values                            
    return(dr)

In [ ]:
path_list = ["data/reason/reason/abortion/*.rsn"]
topic = "Abortion"

dr_abort = create_debate_df(path_list, topic)
dr_abort.text_source_domain.value_counts()

In [ ]:
path_list = ["data/reason/reason/marijuana/*.rsn"]
topic = "Marijuana"

dr_marijuana = create_debate_df(path_list, topic)
dr_marijuana.text_source_domain.value_counts()

In [ ]:
path_list = ["data/reason/reason/gayRights/*.rsn"]
topic = "Gay Marriage"

dr_gay = create_debate_df(path_list, topic)
dr_gay.text_source_domain.value_counts()

In [ ]:
dr = pd.concat([dr_abort, dr_marijuana, dr_gay], ignore_index=True)
dr = args_df.merge(dr, left_on=['topic'], right_on=['text_topic'])
dr['text_processed'], dr['text_processed_size'] = preprocess_sentence(dr.text)

# Distance computation - WMD

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
model = KeyedVectors.load_word2vec_format('data/model.bin', binary=True)
model.init_sims(replace=True)

In [ ]:
from tqdm import tqdm
import math
tqdm.pandas()

def wmd_computation(df):
    df['wmd'] = df.progress_apply(lambda x: model.wmdistance(x.argument_processed.split(), x.text_processed.split()), axis=1)
    df = df.query("wmd != "+str(math.inf))
    df = df.assign(wmd_norm = (df.wmd - df.wmd.min()) / (df.wmd.max() - df.wmd.min()))
    return(df)

### Arguments x Paragraph distance 

In [ ]:
#args_prg_dist = wmd_computation(args_prg)

In [ ]:
#args_prg_dist.to_csv("../data/args_prg_dist.csv", index_label=False)

In [ ]:
args_prg_dist = pd.read_csv("data/args_prg_dist.csv")

### Arguments x Documents distance 

In [ ]:
#sampled_data = args_docs.sample(4000)
#args_docs_dist = wmd_computation(sampled_data)

In [ ]:
#args_docs_dist.to_csv("../data/sampled_args_docs_dist.csv", index_label=False)

In [ ]:
args_docs_dist = pd.read_csv("data/sampled_args_docs_dist.csv")

In [ ]:
#count = 0
#idx = []
#for txt in args_docs_dist.text:
#    count += 1
#    if ((topic == "Marijuana") and ("medic" in txt)):        
#        idx.append(count)
        
#medic_marijuana_dist = args_docs_dist.iloc[idx]
#abortion_dist = args_docs_dist[args_docs_dist.text_topic == "Abortion"]
#args_docs_dist = pd.concat([medic_marijuana_dist, abortion_dist])

### Arguments x Debate Reasons distance 

In [ ]:
#args_dr_dist = wmd_computation(dr)
#args_dr_dist.to_csv("data/args_dr_dist.csv", index_label=False)
args_dr_dist = pd.read_csv("data/args_dr_dist.csv")

# Distance Computation - Opinion Distance

In [5]:
import sys  
sys.path.append("../Opinion-Distance/src/")  
from run_opinion_measure import get_opinion_distance, get_files_and_ground_truth, parse_args

In [6]:
import codecs
cmd_args = ["--path", "../Opinion-Distance/dataset/CiviQ_Seanad/", "--embedding_strategy", "word2vec", "--semantic_threshold", "0.6", "--baselines", "False"]
args = parse_args(cmd_args)

docs_path = "tagme_docs"
full_path_files, ground_truth = None, None

if "CiviQ_Seanad" in args.path:
    Aspect = ["Abolish", "Do not Abolish", "Democracy"]
    full_path_files, ground_truth = get_files_and_ground_truth(args.path, Aspect, docs_path="docs")

filename = ("%s/dist_mats/OD_embedding_strategy_%s.txt") % (args.path, args.embedding_strategy)
fp = codecs.open(filename.replace("dist_mats", "results"), "w")

if args.baselines:
    tf_idf_result(args.path + "dist_mats/tf_idf.txt")

get_opinion_distance(filename=filename, filepath=fp, nfp_available=False, full_path_files=full_path_files, args=args, ground_truth=ground_truth)

Finding opinion distance ../Opinion-Distance/dataset/CiviQ_Seanad//dist_mats/OD_embedding_strategy_word2vec.txt
Representation time  2.161384344100952
Opinion time 0.02850651741027832
[0, 8, 0, 8, 8, 0, 8]


/home/allan/.local/lib/python3.7/site-packages/sklearn/cluster/_spectral.py:482: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


# WEAT

In [ ]:
from mlxtend.evaluate import permutation_test

def weat(dist_df):

    # mean distance of each text to each argument (argument - con/pro) set
    sim_text_args = dist_df.groupby(["text","text_topic","text_title","text_source_domain","argument_orient","text_source_stance"]).wmd_norm.agg(['mean']).reset_index()
    sim_text_args = sim_text_args.rename(columns={"mean":"mean_dist"})

    # difference of distance of each paragraph to the attribute sets
    con_args = sim_text_args[sim_text_args.argument_orient == "Con"]
    pro_args = sim_text_args[sim_text_args.argument_orient == "Pro"]
    sim_text_args = pro_args.merge(con_args, on=["text","text_topic","text_title","text_source_domain","text_source_stance"]).drop(columns=["argument_orient_x", "argument_orient_y"])
    sim_text_args = sim_text_args.rename(columns={"mean_dist_x":"pro_dist","mean_dist_y":"con_dist"})
    sim_text_args["diff_dist"] = sim_text_args.pro_dist - sim_text_args.con_dist

    topics = sim_text_args.text_topic.unique()

    significance_test_array = []

    for topic in topics:
        topic_sources = sim_text_args[sim_text_args.text_topic == topic]
        sources = topic_sources.text_source_domain.unique()
        
        for source in sources:
            source_a = topic_sources[(topic_sources.text_source_domain == sources[0])]
            source_b = topic_sources[(topic_sources.text_source_domain == sources[1])]

            p_value = permutation_test(source_b.diff_dist, source_a.diff_dist,
                                       method='approximate', num_rounds=1000)
            
            interpret = source_b.text_source_domain.iloc[0]+"-Pro x "+source_a.text_source_domain.iloc[0]+"-Con"
            if source_b.diff_dist.mean() > source_a.diff_dist.mean():
                interpret = source_b.text_source_domain.iloc[0]+"-Con x "+source_a.text_source_domain.iloc[0]+"-Pro"
            
            groundtruth = source_b.text_source_domain.iloc[0] + "-" + source_b.text_source_stance.iloc[0] + " x " + source_a.text_source_domain.iloc[0] +"-"+ source_a.text_source_stance.iloc[0]
    
        significance_test_array.append([topic, p_value, groundtruth, interpret, source_b.diff_dist.mean(), source_a.diff_dist.mean()])
    
    significance_test_df = pd.DataFrame(significance_test_array, columns=['topic', 'p_value', 'ground_truth', 'res_interpretation', 'cons_mean_dif', 'rwiki_mean_dif'])
    
    return(significance_test_df)

In [ ]:
weat(args_prg_dist)

In [ ]:
weat(args_docs_dist)

In [ ]:
weat(args_dr_dist)